# Pulling Team Statistics for Comparison 

Using Football-API, I'll be pulling data about team statistics to create a Tableau dashboard comparing two teams' season statistics. Since I'm starting on the free plan, only team statistics from the 2021, 2022, and 2023 seasons can be pulled. The purpose for this notebook is using the API to pull data and transform it into a pandas dataframe to be exported for Tableau use (in a xlsx file or csv file to be determined in the future). 

In [1]:
# importing libraries
import requests
import numpy as np
import pandas as pd

To start I'll be pulling team statistics from the Premier League during the 2021-2022 season (aka the 2021 season). Thus during the API requests the parameters to add will be `league=39` (the league id for the English Premier League), `season=2021` (trivial), and `team=___` depending on what the team's id number is (which can be found in the API dashboard).

In [ ]:
# code chunk for the API request
# the documentation for Football-API recommends using this endpoint only once a week for each team
url = "https://v3.football.api-sports.io/countries"

payload = {}
headers = {
  'x-rapidapi-key': '01914987bf14c6a4b868a79dfe489f07',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

response = requests.request("GET", url, headers = headers, data = payload)

print(response.json())